# Agentic Knowledge Discovery: Unlocking Enterprise Insights with MongoDB Vector Search, LangChain, and LangGraph

-------------

In this comprehensive notebook, you'll gain hands-on expertise in:

- **Building Powerful RAG Systems**: Implement sophisticated Retrieval Augmented Generation using LangChain and MongoDB's versatile data capabilities
- **Advanced Retrieval Strategies**: Master multi retrieval techniques spanning text, vector embeddings, graph relationships, and hybrid approaches for optimal context retrieval
- **Structured Workflows with LangGraph**: Design predictable, deterministic pipelines that ensure reliability and control in complex AI applications
- **Autonomous AI Agents**: Create flexible, agentic systems that can reason, plan, and adapt to accomplish complex tasks
- **Tool Integration at Scale**: Develop frameworks for expanding agent capabilities through seamless tool integration and orchestration
- **Performance Analytics**: Implement comprehensive monitoring and observability using LangSmith to optimize, debug and ensure production-ready AI systems

## Use Case Overview

In today’s fast-paced enterprise environment, organizations generate vast amounts of diverse data—ranging from support FAQs and project documentation to employee directories and expertise profiles. While this information resides in centralized systems, it often remains siloed and difficult for users to navigate using traditional keyword search.

Imagine a network engineer seeking details about a legacy infrastructure upgrade project, a project manager looking for expert collaborators, or a new hire trying to find answers to common onboarding questions. Each query spans multiple data sources, requiring the user to switch contexts, recall precise search terms, or rely on tribal knowledge.

This solution transforms Enterprise Knowledge Discovery by:

- Bridging silos: Unifying diverse collections (FAQs, projects, employee records) under a single semantic index.

- Empowering users: Allowing natural-language queries and conversational exploration across data domains.

- Automating workflows: Leveraging agentic orchestration to decide how and when to retrieve, refine, or chain search steps.

Built on MongoDB Atlas Vector Search for high-performance semantic retrieval, LangChain and LangGraph for embedding and workflow orchestration, this approach delivers an intuitive, intelligent, and scalable knowledge platform tailored for large enterprises.

**Objective**:

Enable enterprise users to query and explore organizational knowledge across FAQs, project details, and employee expertise in natural language.

**Key Benefits:**

- Reduced time-to-insight: Semantic search surfaces relevant results even when keywords differ.

- Contextual reasoning: Agents chain multi-step queries (e.g., “Which engineer led Project P123?”).

- Scalable architecture: Easily extend to new data sources (Confluence, emails, design documents).

**Key Components:**

- MongoDB Atlas Vector Search: Dense vector indexing for semantic relevance.

- Voyage AI: State of the art embedding models and rerankers

- LangChain: Embedding pipelines and workflow management.

- LangGraph: Agentic, graph-driven decision making for complex queries.



In [1]:
!pip install -qU openai pymongo voyageai langchain_voyageai langchain_openai langchain-mongodb langgraph-checkpoint-mongodb langchain-core langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1

// List what each of the packages do

In [2]:
import getpass
import os


# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [3]:
# Set your OpenAI API Key
# TODO: Place a link on where openai api key can be obtained
set_env_securely("OPENAI_API_KEY", "Enter your OPENAI API KEY: ")

Enter your OPENAI API KEY: ··········


## Part 0: Synthetic Data Creation

In [4]:
import json
from typing import Any, List, Optional

from pydantic import BaseModel, Field


# Define Pydantic data models for datasets
class FAQ(BaseModel):
    faq_id: str = Field(..., description="Unique FAQ identifier")
    question: str = Field(..., description="FAQ question text")
    answer: str = Field(..., description="FAQ answer text")
    tags: List[str] = Field(default_factory=list, description="Related tags")


class Project(BaseModel):
    project_id: str = Field(..., description="Unique project identifier")
    name: str = Field(..., description="Project name")
    description: str = Field(..., description="Detailed project description")
    status: str = Field(..., description="Current project status")
    start_date: str = Field(..., description="ISO format start date")
    end_date: str = Field(..., description="ISO format end date")

    # Team information
    project_manager: str = Field(..., description="Employee ID of project manager")
    team_members: List[str] = Field(..., description="List of employee IDs")

    # Technical details
    technologies: List[str] = Field(..., description="Technology stack")
    skills_required: List[str] = Field(..., description="Required skills")

    # Project relationships
    dependencies: List[str] = Field(
        default_factory=list, description="Dependent project IDs"
    )
    related_projects: List[str] = Field(
        default_factory=list, description="Related project IDs"
    )


class Employee(BaseModel):
    emp_id: str = Field(..., description="Unique employee identifier")
    name: str = Field(..., description="Employee full name")
    role: str = Field(..., description="Employee role/title")
    department: str = Field(..., description="Department name")
    skills: List[str] = Field(default_factory=list, description="List of skills")
    bio: Optional[str] = Field(None, description="Short professional biography")
    manager: Optional[str] = Field(None, description="Employee ID of manager")
    start_date: str = Field(..., description="ISO format start date")
    end_date: str = Field(..., description="ISO format end date")

    # Project relationships
    current_projects: List[str] = Field(
        default_factory=list, description="Current project IDs"
    )
    past_projects: List[str] = Field(
        default_factory=list, description="Past project IDs"
    )

    # Team relationships
    mentors: List[str] = Field(
        default_factory=list, description="Employee IDs of mentors"
    )
    mentees: List[str] = Field(
        default_factory=list, description="Employee IDs of mentees"
    )
    frequent_collaborators: List[str] = Field(
        default_factory=list, description="Frequent collaborators"
    )


class KnowledgeAsset(BaseModel):
    asset_id: str = Field(..., description="Unique knowledge asset ID")
    title: str = Field(..., description="Title of the knowledge asset")
    content: str = Field(..., description="Content or description")
    type: str = Field(..., description="Type (documentation, best_practice, solution)")
    author: str = Field(..., description="Employee ID of author")
    creation_date: str = Field(..., description="ISO creation date")
    tags: List[str] = Field(default_factory=list, description="Tags for categorization")
    related_projects: List[str] = Field(
        default_factory=list, description="Related project IDs"
    )
    related_employees: List[str] = Field(
        default_factory=list, description="Related employee IDs"
    )


# Mapping from dataset type to Pydantic model
model_map = {
    "faqs": FAQ,
    "projects": Project,
    "employees": Employee,
    "knowledge_assets": KnowledgeAsset,
}

In [5]:
class DatasetReference:
    """Track generated IDs for cross-referencing between datasets"""

    def __init__(self):
        self.employee_ids = set()
        self.project_ids = set()
        self.faq_ids = set()
        self.knowledge_asset_ids = set()

In [6]:
data_refs = DatasetReference()

In [7]:
from openai import OpenAI

openai_client = OpenAI()


def generate_synthetic_data_with_refs(
    dataset_type: str, instructions: str, entry: int, data_refs
) -> BaseModel:
    """Generate synthetic data with valid references to other datasets"""

    if dataset_type not in model_map:
        raise ValueError(f"Unsupported dataset type: {dataset_type}")

    selected_model = model_map[dataset_type]
    current_id = f"{dataset_type}-{entry}"

    # Build reference context based on dataset type
    if dataset_type == "employees":
        data_refs.employee_ids.add(current_id)
        reference_context = f"""
        Valid reference IDs:
        - Employee IDs: {list(data_refs.employee_ids) if data_refs.employee_ids else 'None yet'}
        - Project IDs: {list(data_refs.project_ids) if data_refs.project_ids else 'None yet'}

        Instructions:
        - For mentors/mentees: Use existing employee IDs from the list above
        - For current_projects/past_projects: Use existing project IDs
        - Leave lists empty if no valid IDs are available yet
        """

    elif dataset_type == "projects":
        data_refs.project_ids.add(current_id)
        reference_context = f"""
        Valid reference IDs:
        - Employee IDs: {list(data_refs.employee_ids)}
        - Project IDs: {list(data_refs.project_ids) if data_refs.project_ids else 'None yet'}

        Instructions:
        - For team_members/project_manager: Use employee IDs from the list above
        - For dependencies/related_projects: Use project IDs (leave empty for first few projects)
        - Ensure all referenced IDs exist in the lists above
        """

    elif dataset_type == "knowledge_assets":
        data_refs.knowledge_asset_ids.add(current_id)
        reference_context = f"""
        Valid reference IDs:
        - Employee IDs: {list(data_refs.employee_ids)}
        - Project IDs: {list(data_refs.project_ids)}

        Instructions:
        - For author: Use one employee ID from the list above
        - For related_projects: Use valid project IDs
        - For related_employees: Use valid employee IDs
        """

    else:  # FAQs don't need references
        reference_context = "This entity type doesn't need ID references."

    # Enhanced instructions with reference context
    full_instructions = f"""
    {instructions}

    REFERENCE CONTEXT:
    {reference_context}

    IMPORTANT: Only use IDs from the valid reference lists above.
    """

    # Generate the data
    response = openai_client.responses.parse(
        model="gpt-4.1",
        input=f"Generate a synthetic {dataset_type} record with the id '{current_id}'. {full_instructions}",
        text_format=selected_model,
    )

    return response.output_parsed

In [8]:
from tqdm import tqdm


def generate_all_synthetic_data():
    """Generate all synthetic datasets with proper ID references"""

    # Clear previous references
    global data_refs
    data_refs = DatasetReference()

    all_datasets = {}

    # 1. Generate Employees first (they're referenced by others)
    print("Generating Employees...")
    employee_dataset_generation_instruction = """
    Generate Employee record for a telecommunications company:
    - Include roles like network engineer, system administrator, project manager
    - Add mentorship relationships (using valid employee IDs)
    - Add project assignments (using valid project IDs)
    - Include bio and skills relevant to telecom industry
    """

    number_of_employee_datapoints = 10
    employee_datapoints = []

    for i in tqdm(
        range(number_of_employee_datapoints), desc="Generating Employee datapoints"
    ):
        generated_employee = generate_synthetic_data_with_refs(
            "employees",
            employee_dataset_generation_instruction,
            entry=i,
            data_refs=data_refs,
        )
        employee_datapoints.append(generated_employee)

    all_datasets["employees"] = employee_datapoints

    # 2. Generate Projects (referenced by knowledge assets)
    print("Generating Projects...")
    project_dataset_generation_instruction = """
    Generate Project record for telecommunications company:
    - Include valid team member IDs from employees
    - Add project dependencies (using valid project IDs)
    - Include technology stacks relevant to telecom
    - Reference valid project manager from employees
    """

    number_of_project_datapoints = 8
    project_datapoints = []

    for i in tqdm(
        range(number_of_project_datapoints), desc="Generating Project datapoints"
    ):
        generated_project = generate_synthetic_data_with_refs(
            "projects",
            project_dataset_generation_instruction,
            entry=i,
            data_refs=data_refs,
        )
        project_datapoints.append(generated_project)

    all_datasets["projects"] = project_datapoints

    # 3. Generate FAQs (no ID references needed)
    print("Generating FAQs...")
    faq_dataset_generation_instruction = """
    Generate FAQs based on a company all hands at a telecoms company like Cisco.
    """

    number_of_faq_datapoints = 5
    faq_datapoints = []

    for i in tqdm(range(number_of_faq_datapoints), desc="Generating FAQ datapoints"):
        generated_faq = generate_synthetic_data_with_refs(
            "faqs", faq_dataset_generation_instruction, entry=i, data_refs=data_refs
        )
        faq_datapoints.append(generated_faq)

    all_datasets["faqs"] = faq_datapoints

    # 4. Generate Knowledge Assets (reference both employees and projects)
    print("Generating Knowledge Assets...")
    knowledge_asset_dataset_generation_instruction = """
    Generate Knowledge Asset record:
    - Use valid author employee ID
    - Reference valid related projects
    - Include valid related employees
    - Document technical procedures and best practices
    """

    number_of_knowledge_asset_datapoints = 6
    knowledge_asset_datapoints = []

    for i in tqdm(
        range(number_of_knowledge_asset_datapoints),
        desc="Generating Knowledge Asset datapoints",
    ):
        generated_knowledge_asset = generate_synthetic_data_with_refs(
            "knowledge_assets",
            knowledge_asset_dataset_generation_instruction,
            entry=i,
            data_refs=data_refs,
        )
        knowledge_asset_datapoints.append(generated_knowledge_asset)

    all_datasets["knowledge_assets"] = knowledge_asset_datapoints

    return all_datasets

In [9]:
synthetic_datasets = generate_all_synthetic_data()

Generating Employees...


Generating Employee datapoints: 100%|██████████| 10/10 [00:30<00:00,  3.01s/it]


Generating Projects...


Generating Project datapoints: 100%|██████████| 8/8 [01:00<00:00,  7.60s/it]


Generating FAQs...


Generating FAQ datapoints: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]


Generating Knowledge Assets...


Generating Knowledge Asset datapoints: 100%|██████████| 6/6 [00:23<00:00,  3.88s/it]


In [10]:
import pandas as pd


def export_datapoints_to_json(
    datapoints: List[Any],
    dataset_type: str,
    output_dir: str = "synthetic_data",
    indent: int = 2,
) -> str:
    """
    Exports a list of Pydantic model instances (or dict-like objects) to a JSON file.

    Args:
      datapoints (List[Any]): List of Pydantic instances or dicts.
      dataset_type (str): Identifier for the dataset (e.g., 'faqs', 'projects').
      indent (int): Number of spaces for JSON indentation.

    Returns:
      str: The full path to the saved JSON file.
    """

    # Convert models to dicts
    list_of_dicts = [
        dp.model_dump()
        if hasattr(dp, "model_dump")
        else getattr(dp, "dict", lambda: dp)()
        for dp in datapoints
    ]

    # Create DataFrame
    df = pd.DataFrame(list_of_dicts)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Build filename and path
    filename = f"{dataset_type}_datapoints.json"
    output_path = os.path.join(output_dir, filename)

    # Export to JSON
    df.to_json(output_path, orient="records", indent=indent)

    print(f"Saved {len(df)} records to {output_path}")
    return output_path

In [11]:
export_datapoints_to_json(datapoints=synthetic_datasets["faqs"], dataset_type="faqs")

export_datapoints_to_json(
    datapoints=synthetic_datasets["employees"], dataset_type="employees"
)

export_datapoints_to_json(
    datapoints=synthetic_datasets["projects"], dataset_type="projects"
)

export_datapoints_to_json(
    datapoints=synthetic_datasets["knowledge_assets"], dataset_type="knowledge_assets"
)

Saved 5 records to synthetic_data/faqs_datapoints.json
Saved 10 records to synthetic_data/employees_datapoints.json
Saved 8 records to synthetic_data/projects_datapoints.json
Saved 6 records to synthetic_data/knowledge_assets_datapoints.json


'synthetic_data/knowledge_assets_datapoints.json'

## Part 1: Data Loading, Cleaning and Preparation

In [12]:
set_env_securely("VOYAGE_API_KEY", "Enter your VOYAGE AI API key: ")

Enter your VOYAGE AI API key: ··········


In [13]:
# TODO: Change to hugging face
employees_data_df = pd.read_json("synthetic_data/employees_datapoints.json")
faqs_data_df = pd.read_json("synthetic_data/faqs_datapoints.json")
projects_data_df = pd.read_json("synthetic_data/projects_datapoints.json")
knowledge_assets_data_df = pd.read_json(
    "synthetic_data/knowledge_assets_datapoints.json"
)

In [14]:
employees_data_df.head()

,emp_id,name,role,department,skills,bio,manager,start_date,end_date,current_projects,past_projects,mentors,mentees,frequent_collaborators
0,employees-0,Jordan Ellis,Network Engineer,Network Operations,"[Network design, Routing and switching, Fiber ...",Jordan has over 7 years of experience designin...,None,2017-03-15,,[],[],[],[],[]
1,employees-1,Sofia Martinez,Network Engineer,Network Operations,"[Network Design, Routing & Switching, VoIP, Fi...",Sofia Martinez is a Network Engineer specializ...,employees-0,2022-04-10,,[],[],[employees-0],[],[employees-0]
2,employees-2,Jasmine Chen,Network Engineer,Network Operations,"[IP networking, Routing protocols, Switch conf...",Jasmine is a talented Network Engineer special...,employees-0,2022-03-15,,[],[],[employees-1],[],"[employees-0, employees-1]"
3,employees-3,Jessica Lin,Network Engineer,Network Operations,"[Network Design, Troubleshooting, Routing & Sw...",Jessica Lin is an experienced Network Engineer...,employees-1,2021-09-13,,[],[],[employees-0],[employees-2],"[employees-0, employees-1]"
4,employees-4,Sophia Chen,Network Engineer,Network Operations,"[Network Design, Routing & Switching, Fiber Op...",Sophia is a Network Engineer with over 7 years...,employees-2,2021-08-01,,[],[],[employees-1],[employees-3],"[employees-2, employees-0]"


In [15]:
faqs_data_df.head()

,faq_id,question,answer,tags
0,faqs-0,What topics were covered in the recent company...,The recent all-hands meeting covered several k...,"[all-hands, company meeting, telecoms, Cisco, ..."
1,faqs-1,What topics were discussed in the latest compa...,The latest company all hands meeting covered b...,"[company all hands, telecom, Cisco, business u..."
2,faqs-2,What topics are typically covered in a telecom...,All hands meetings at a telecom company like C...,"[all hands, meeting, telecom, company updates,..."
3,faqs-3,What topics were covered in the most recent co...,"In the most recent company all hands, we discu...","[company all hands, telecom, meeting recap, in..."
4,faqs-4,What topics are typically covered during a tel...,"During a telecoms company all hands, topics us...","[all hands, telecom, company meeting, Cisco, l..."


### Generating emebdding for datapoints

In [16]:
from typing import Optional

import voyageai

VOYAGE_AI_EMBEDDING_MODEL = "voyage-3-large"
VOYAGE_AI_EMBEDDING_MODEL_DIMENSION = 1024

# Initialize the Voyage AI client.
voyageai_client = voyageai.Client()


def get_embedding(text, task_prefix="document"):
    """
    Generate embeddings for a text string with a task-specific prefix using the voyage-3-large model.

    Parameters:
      text (str): The input text to be embedded.
      task_prefix (str): A prefix describing the task; this is prepended to the text.

    Returns:
      list: The embedding vector as a list of floats (or ints if another output_dtype is chosen).
    """
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    # Call the Voyage API to generate the embedding.
    # Here, we wrap the text in a list since the API expects a list of texts.
    # Default output embedding: 1024
    result = voyageai_client.embed(
        [text], model=VOYAGE_AI_EMBEDDING_MODEL, input_type=task_prefix
    )

    # Return the first embedding from the result.
    return result.embeddings[0]

In [17]:
from tqdm import tqdm


def generate_employee_embedding(employee_row):
    """
    Generate an embedding for an employee by concatenating relevant fields.

    Parameters:
        employee_row (pandas.Series): A row from the employees DataFrame

    Returns:
        list: The embedding vector
    """
    try:
        # Ensure each field exists and handle possible missing values
        name = employee_row.get("name", "")
        role = employee_row.get("role", "")
        department = employee_row.get("department", "")

        # Handle skills which should be a list
        skills = employee_row.get("skills", [])
        if not isinstance(skills, list):
            skills = [] if pd.isna(skills) else [str(skills)]

        # Handle bio which might be None/NaN
        bio = employee_row.get("bio", "")
        if pd.isna(bio):
            bio = ""

        # Concatenate relevant fields with spaces in between
        concatenated_text = (
            f"Name: {name} "
            f"Role: {role} "
            f"Department: {department} "
            f"Skills: {', '.join(skills)} "
            f"Bio: {bio}"
        )

        # Generate and return the embedding
        return get_embedding(concatenated_text)
    except Exception as e:
        print(
            f"Error generating embedding for {employee_row.get('emp_id', 'unknown')}: {e}"
        )
        # Return empty list or None to indicate error
        return []

In [18]:
def generate_faq_embedding(faq_row):
    """
    Generate an embedding for an FAQ by concatenating question and answer fields.

    Parameters:
        faq_row (pandas.Series): A row from the FAQs DataFrame

    Returns:
        list: The embedding vector
    """
    try:
        # Ensure each field exists and handle possible missing values
        question = faq_row.get("question", "")
        answer = faq_row.get("answer", "")

        # Concatenate question and answer fields with space in between
        concatenated_text = f"Question: {question} Answer: {answer}"

        # Generate and return the embedding
        return get_embedding(concatenated_text)
    except Exception as e:
        print(f"Error generating embedding for {faq_row.get('faq_id', 'unknown')}: {e}")
        # Return empty list or None to indicate error
        return []

In [19]:
def generate_knowledge_asset_embedding(knowledge_asset_row):
    """
    Generate an embedding for a knowledge asset by concatenating title and content fields.

    Parameters:
        knowledge_asset_row (pandas.Series): A row from the Knowledge Assets DataFrame

    Returns:
        list: The embedding vector
    """

    try:
        # Ensure each field exists and handle possible missing values
        title = knowledge_asset_row.get("title", "")
        content = knowledge_asset_row.get("content", "")

        # Concatenate title and content fields with space in between
        concatenated_text = f"Title: {title} Content: {content}"

        # Generate and return the embedding
        return get_embedding(concatenated_text)
    except Exception as e:
        print(
            f"Error generating embedding for {knowledge_asset_row.get('asset_id', 'unknown')}: {e}"
        )
        # Return empty list or None to indicate error
        return []

In [20]:
def generate_project_embedding(project_row):
    """
    Generate an embedding for a project by concatenating name and description fields.

    Parameters:
        project_row (pandas.Series): A row from the Projects DataFrame

    Returns:
        list: The embedding vector
    """
    try:
        # Ensure each field exists and handle possible missing values
        name = project_row.get("name", "")
        description = project_row.get("description", "")
        status = project_row.get("status", "")

        # Concatenate name, description,
        concatenated_text = f"Name: {name} Description: {description} Status: {status}"

        # Generate and return the embedding
        return get_embedding(concatenated_text)
    except Exception as e:
        print(
            f"Error generating embedding for {project_row.get('project_id', 'unknown')}: {e}"
        )
        # Return empty list or None to indicate error
        return []

In [21]:
# Apply the function to each row in the employees DataFrame
tqdm.pandas(desc="Generating employee embeddings")
employees_data_df["embedding"] = employees_data_df.progress_apply(
    generate_employee_embedding, axis=1
)

Generating employee embeddings: 100%|██████████| 10/10 [00:02<00:00,  4.44it/s]


In [22]:
employees_data_df.head()

,emp_id,name,role,department,skills,bio,manager,start_date,end_date,current_projects,past_projects,mentors,mentees,frequent_collaborators,embedding
0,employees-0,Jordan Ellis,Network Engineer,Network Operations,"[Network design, Routing and switching, Fiber ...",Jordan has over 7 years of experience designin...,None,2017-03-15,,[],[],[],[],[],"[-0.08703910559415817, 0.02716110460460186, -0..."
1,employees-1,Sofia Martinez,Network Engineer,Network Operations,"[Network Design, Routing & Switching, VoIP, Fi...",Sofia Martinez is a Network Engineer specializ...,employees-0,2022-04-10,,[],[],[employees-0],[],[employees-0],"[-0.07327310740947723, 0.05744690075516701, -0..."
2,employees-2,Jasmine Chen,Network Engineer,Network Operations,"[IP networking, Routing protocols, Switch conf...",Jasmine is a talented Network Engineer special...,employees-0,2022-03-15,,[],[],[employees-1],[],"[employees-0, employees-1]","[-0.07105905562639236, 0.02156638726592064, -0..."
3,employees-3,Jessica Lin,Network Engineer,Network Operations,"[Network Design, Troubleshooting, Routing & Sw...",Jessica Lin is an experienced Network Engineer...,employees-1,2021-09-13,,[],[],[employees-0],[employees-2],"[employees-0, employees-1]","[-0.06169969215989113, 0.036690425127744675, 0..."
4,employees-4,Sophia Chen,Network Engineer,Network Operations,"[Network Design, Routing & Switching, Fiber Op...",Sophia is a Network Engineer with over 7 years...,employees-2,2021-08-01,,[],[],[employees-1],[employees-3],"[employees-2, employees-0]","[-0.074442557990551, 0.033538103103637695, -0...."


In [23]:
# Apply the function to each row in the FAQs DataFrame with tqdm progress bar
tqdm.pandas(desc="Generating FAQ embeddings")
faqs_data_df["embedding"] = faqs_data_df.progress_apply(generate_faq_embedding, axis=1)

Generating FAQ embeddings: 100%|██████████| 5/5 [00:01<00:00,  4.51it/s]


In [24]:
faqs_data_df.head()

,faq_id,question,answer,tags,embedding
0,faqs-0,What topics were covered in the recent company...,The recent all-hands meeting covered several k...,"[all-hands, company meeting, telecoms, Cisco, ...","[-0.02724125236272812, 0.04230331629514694, -0..."
1,faqs-1,What topics were discussed in the latest compa...,The latest company all hands meeting covered b...,"[company all hands, telecom, Cisco, business u...","[-0.04060741141438484, 0.058594949543476105, -..."
2,faqs-2,What topics are typically covered in a telecom...,All hands meetings at a telecom company like C...,"[all hands, meeting, telecom, company updates,...","[-0.016593847423791885, 0.04799298569560051, -..."
3,faqs-3,What topics were covered in the most recent co...,"In the most recent company all hands, we discu...","[company all hands, telecom, meeting recap, in...","[-0.02428397536277771, 0.05805167555809021, -0..."
4,faqs-4,What topics are typically covered during a tel...,"During a telecoms company all hands, topics us...","[all hands, telecom, company meeting, Cisco, l...","[-0.027287932112812996, 0.05719038471579552, -..."


In [25]:
tqdm.pandas(desc="Generating Knowledge Asset embeddings")
knowledge_assets_data_df["embedding"] = knowledge_assets_data_df.progress_apply(
    generate_knowledge_asset_embedding, axis=1
)

Generating Knowledge Asset embeddings: 100%|██████████| 6/6 [00:01<00:00,  4.78it/s]


In [26]:
knowledge_assets_data_df.head()

,asset_id,title,content,type,author,creation_date,tags,related_projects,related_employees,embedding
0,knowledge_assets-0,Continuous Integration and Deployment Best Pra...,This asset documents technical procedures and ...,best_practice,employees-3,2024-06-10T13:00:00Z,"[CI/CD, automation, deployment, devops, techni...","[projects-5, projects-7]","[employees-7, employees-9, employees-3]","[-0.07771030068397522, 0.05420322343707085, 0...."
1,knowledge_assets-1,CI/CD Pipeline Setup and Best Practices,This asset documents the technical procedures ...,best_practice,employees-2,2024-06-10T09:30:00Z,"[CI/CD, Jenkins, Docker, Automation, DevOps, B...","[projects-3, projects-7]","[employees-6, employees-2, employees-9]","[-0.0717896819114685, 0.034540530294179916, 0...."
2,knowledge_assets-2,Continuous Integration and Deployment: Technic...,This document details the technical procedures...,best_practice,employees-3,2024-05-15,"[CI/CD, technical procedures, best practices, ...","[projects-0, projects-4, projects-2]","[employees-7, employees-1, employees-9]","[-0.06140996143221855, 0.04652770236134529, 0...."
3,knowledge_assets-3,Best Practices for CI/CD Pipeline Automation,This document details the technical procedures...,best_practice,employees-7,2024-06-27,"[CI/CD, automation, Jenkins, Docker, best prac...","[projects-3, projects-6]","[employees-2, employees-3, employees-5]","[-0.0732276514172554, 0.057461995631456375, 0...."
4,knowledge_assets-4,CI/CD Pipeline Setup: Technical Procedures and...,This document details the technical procedures...,documentation,employees-4,2024-06-04T10:15:00Z,"[CI/CD, automation, deployment, best_practice,...","[projects-3, projects-7]","[employees-4, employees-7, employees-9]","[-0.08069836348295212, 0.07005231082439423, 0...."


In [27]:
tqdm.pandas(desc="Generating Project embeddings")
projects_data_df["embedding"] = projects_data_df.progress_apply(
    generate_project_embedding, axis=1
)

Generating Project embeddings: 100%|██████████| 8/8 [00:01<00:00,  4.26it/s]


In [28]:
projects_data_df.head()

,project_id,name,description,status,start_date,end_date,project_manager,team_members,technologies,skills_required,dependencies,related_projects,embedding
0,projects-0,NextGen 5G Network Expansion,"This project focuses on the design, deployment...",In Progress,2024-02-01,2025-01-31,employees-4,"[employees-4, employees-7, employees-9, employ...","[5G NR, Core Network (EPC/5GC), Fiber Optics, ...","[Telecommunications Engineering, Project Manag...",[],[],"[-0.08444010466337204, 0.02754852920770645, -0..."
1,projects-1,5G Network Expansion Initiative,This project focuses on expanding 5G wireless ...,Active,2024-05-01,2025-10-31,employees-7,"[employees-7, employees-2, employees-3, employ...","[5G NR (New Radio), Fiber Optic Networking, Ne...","[RF Engineering, Network Design, Project Manag...",[projects-0],[projects-0],"[-0.06901782751083374, 0.040653444826602936, -..."
2,projects-2,5G Network Expansion Initiative,This project focuses on expanding and upgradin...,In Progress,2024-01-15,2025-03-30,employees-4,"[employees-4, employees-6, employees-9, employ...","[5G NR, SDN (Software Defined Networking), NFV...","[Network Engineering, Telecom Protocols, SDN C...",[projects-0],[projects-1],"[-0.06328871846199036, 0.03281654044985771, -0..."
3,projects-3,5G Network Infrastructure Upgrade,This project involves upgrading existing telec...,In Progress,2024-03-01,2025-03-31,employees-3,"[employees-3, employees-2, employees-5, employ...","[5G NR, Fiber Optic Networking, SDN (Software ...","[RF Engineering, Network Design, Telecommunica...",[projects-1],[projects-2],"[-0.08185975253582001, 0.043286748230457306, -..."
4,projects-4,5G Network Optimization Initiative,This project focuses on optimizing the existin...,Active,2024-04-10,2025-02-28,employees-7,"[employees-3, employees-4, employees-7, employ...","[5G NR, SDN (Software Defined Networking), Net...","[Telecommunications Engineering, Network Optim...","[projects-2, projects-1]","[projects-0, projects-3]","[-0.0686565637588501, 0.037224143743515015, -0..."


## Part 2: Database Connection, Collection and Indexes

### Connecting to MongoDB

MongoDB acts as both an operational and a vector database for the RAG system.
MongoDB Atlas specifically provides a database solution that efficiently stores, queries and retrieves vector embeddings.

#### Setup

To use MongoDB as a toolbox, you will need to complete the following steps:

1. Register for a MongoDB Account:
   - Go to the MongoDB website (https://www.mongodb.com/cloud/atlas/register).
   - Click on the "Try Free" or "Get Started Free" button.
   - Fill out the registration form with your details and create an account.

2. Create a [MongoDB Cluster](https://www.mongodb.com/docs/atlas/tutorial/deploy-free-tier-cluster/#procedure)

3. Set Up [Database Access](https://www.mongodb.com/docs/atlas/security-add-mongodb-users/#add-database-users):
   - In the left sidebar, click on "Database Access" under "Security".
   - Click "Add New Database User".
   - Create a username and a strong password. Save these credentials securely.
   - Set the appropriate permissions for the user (e.g., "Read and write to any database").

4. Configure Network Access:
   - In the left sidebar, click on "Network Access" under "Security".
   - Click "Add IP Address".
   - To allow access from anywhere (not recommended for production), enter 0.0.0.0/0.
   - For better security, whitelist only the specific IP addresses that need access.

5. Follow MongoDB’s [steps to get the connection](https://www.mongodb.com/docs/manual/reference/connection-string/) string from the Atlas UI. After setting up the database and obtaining the Atlas cluster connection URI, securely store the URI within your development environment.


In [29]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

Enter your MongoDB URI: ··········


In [30]:
import pymongo


def get_mongo_client(mongo_uri):
    """Establish and validate connection to the MongoDB."""

    client = pymongo.MongoClient(
        mongo_uri, appname="devrel.showcase.agents.knowledge_discovery.python"
    )

    # Validate the connection
    ping_result = client.admin.command("ping")
    if ping_result.get("ok") == 1.0:
        # Connection successful
        print("Connection to MongoDB successful")
        return client
    else:
        print("Connection to MongoDB failed")
    return None

In [31]:
DB_NAME = "enterprise_knowledge_discovery"
db_client = get_mongo_client(os.environ.get("MONGODB_URI"))
db = db_client[DB_NAME]

Connection to MongoDB successful


### Create collections

In [32]:
# Collection names
EMPLOYEES_COLLECTION = "employees"
FAQS_COLLECTION = "faqs"
KNOWLEDGE_ASSETS_COLLECTION = "knowledge_assets"
PROJECTS_COLLECTION = "projects"

EMPLOYEES_VECTOR_INDEX_NAME = "employees_vector_search_index"
FAQS_VECTOR_INDEX_NAME = "faqs_vector_search_index"
KNOWLEDGE_ASSETS_VECTOR_INDEX_NAME = "knowledge_assets_vector_search_index"
PROJECT_VECTOR_INDEX_NAME = "projects_vector_search_index"

EMPLOYEES_SEARCH_INDEX_NAME = "employees_text_search_index"
FAQS_SEARCH_INDEX_NAME = "faqs_text_search_index"
KNOWLEDGE_ASSETS_SEARCH_INDEX_NAME = "knowledge_assets_text_search_index"
PROJECTS_SEARCH_INDEX_NAME = "projects_text_search_index"

VECTOR_DIMENSION = 1024

In [33]:
# Create collections with validation if they don't exist
def create_collections():
    # Get list of existing collections
    existing_collections = db.list_collection_names()
    print(f"Existing collections: {existing_collections}")

    # Create employees collection with schema validation if it doesn't exist
    if EMPLOYEES_COLLECTION not in existing_collections:
        db.create_collection(
            EMPLOYEES_COLLECTION,
            validator={
                "$jsonSchema": {
                    "bsonType": "object",
                    "required": [
                        "emp_id",
                        "name",
                        "role",
                        "department",
                        "start_date",
                        "end_date",
                    ],
                    "properties": {
                        "emp_id": {"bsonType": "string"},
                        "name": {"bsonType": "string"},
                        "role": {"bsonType": "string"},
                        "department": {"bsonType": "string"},
                        "skills": {"bsonType": "array"},
                        "bio": {"bsonType": ["string", "null"]},
                        "manager": {"bsonType": ["string", "null"]},
                        "current_projects": {"bsonType": "array"},
                        "past_projects": {"bsonType": "array"},
                        "mentors": {"bsonType": "array"},
                        "mentees": {"bsonType": "array"},
                        "frequent_collaborators": {"bsonType": "array"},
                        "start_date": {"bsonType": "string"},
                        "end_date": {"bsonType": "string"},
                        "embedding": {"bsonType": "array"},
                    },
                }
            },
            validationLevel="moderate",
        )
        print(f"Created {EMPLOYEES_COLLECTION} collection with schema validation")

    # Create FAQs collection with schema validation if it doesn't exist
    if FAQS_COLLECTION not in existing_collections:
        db.create_collection(
            FAQS_COLLECTION,
            validator={
                "$jsonSchema": {
                    "bsonType": "object",
                    "required": [
                        "faq_id",
                        "question",
                        "answer",
                    ],
                    "properties": {
                        "faq_id": {"bsonType": "string"},
                        "question": {"bsonType": "string"},
                        "answer": {"bsonType": "string"},
                        "tags": {"bsonType": "array"},
                        "embedding": {"bsonType": "array"},
                    },
                }
            },
            validationLevel="moderate",
        )
        print(f"Created {FAQS_COLLECTION} collection with schema validation")

    # Create Projects collection with schema validation
    if PROJECTS_COLLECTION not in existing_collections:
        db.create_collection(
            PROJECTS_COLLECTION,
            validator={
                "$jsonSchema": {
                    "bsonType": "object",
                    "required": ["project_id", "name", "description", "status"],
                    "properties": {
                        "project_id": {"bsonType": "string"},
                        "name": {"bsonType": "string"},
                        "description": {"bsonType": "string"},
                        "status": {"bsonType": "string"},
                        "team_members": {"bsonType": "array"},
                        "technologies": {"bsonType": "array"},
                        "dependencies": {"bsonType": "array"},
                        "related_projects": {"bsonType": "array"},
                        "embedding": {"bsonType": "array"},
                    },
                }
            },
            validationLevel="moderate",
        )
        print(f"Created {PROJECTS_COLLECTION} collection with schema validation")

    # Create Projects collection with schema validation
    if KNOWLEDGE_ASSETS_COLLECTION not in existing_collections:
        db.create_collection(
            KNOWLEDGE_ASSETS_COLLECTION,
            validator={
                "$jsonSchema": {
                    "bsonType": "object",
                    "required": ["asset_id", "title", "content", "type"],
                    "properties": {
                        "asset_id": {"bsonType": "string"},
                        "title": {"bsonType": "string"},
                        "content": {"bsonType": "string"},
                        "type": {"bsonType": "string"},
                        "author": {"bsonType": "string"},
                        "creation_date": {"bsonType": "string"},
                        "last_updated": {"bsonType": "string"},
                        "tags": {"bsonType": "array"},
                        "related_projects": {"bsonType": "array"},
                        "related_employees": {"bsonType": "array"},
                        "relevance_score": {"bsonType": "number"},
                        "embedding": {"bsonType": "array"},
                    },
                }
            },
            validationLevel="moderate",
        )
        print(
            f"Created {KNOWLEDGE_ASSETS_COLLECTION} collection with schema validation"
        )

In [34]:
# Call function to create collections
create_collections()

Existing collections: ['checkpoints', 'employees', 'projects', 'knowledge_assets', 'faqs', 'checkpoint_writes', 'crisis_events']


### Create Indexes

Create the vector search indexes

In [35]:
import time

from pymongo.operations import SearchIndexModel


# Create vector search index if it doesn't exist
def create_vector_search_index(collection, vector_index_name):
    # Check if index already exists
    try:
        existing_indexes = collection.list_search_indexes()
        for index in existing_indexes:
            if index["name"] == vector_index_name:
                print(f"Vector search index '{vector_index_name}' already exists.")
                return
    except Exception as e:
        print(f"Could not list search indexes: {e}")
        return

    # Create vector search index
    search_index_model = SearchIndexModel(
        definition={
            "fields": [
                {
                    "type": "vector",
                    "path": "embedding",
                    "numDimensions": VECTOR_DIMENSION,
                    "similarity": "cosine",
                }
            ]
        },
        name=vector_index_name,
        type="vectorSearch",
    )

    try:
        result = collection.create_search_index(model=search_index_model)
        print(f"New search index named '{result}' is building.")
    except Exception as e:
        print(f"Error creating vector search index: {e}")
        return

    # Wait for initial sync to complete
    print(
        f"Polling to check if the index '{result}' is ready. This may take up to a minute."
    )
    predicate = lambda index: index.get("queryable") is True

    while True:
        try:
            indices = list(collection.list_search_indexes(result))
            if indices and predicate(indices[0]):
                break
            time.sleep(5)
        except Exception as e:
            print(f"Error checking index readiness: {e}")
            time.sleep(5)

    print(f"{result} is ready for querying.")

In [36]:
create_vector_search_index(db[EMPLOYEES_COLLECTION], EMPLOYEES_VECTOR_INDEX_NAME)
create_vector_search_index(db[FAQS_COLLECTION], FAQS_VECTOR_INDEX_NAME)
create_vector_search_index(
    db[KNOWLEDGE_ASSETS_COLLECTION], KNOWLEDGE_ASSETS_VECTOR_INDEX_NAME
)
create_vector_search_index(db[PROJECTS_COLLECTION], PROJECT_VECTOR_INDEX_NAME)

Vector search index 'employees_vector_search_index' already exists.
Vector search index 'faqs_vector_search_index' already exists.
Vector search index 'knowledge_assets_vector_search_index' already exists.
Vector search index 'projects_vector_search_index' already exists.


Create the search indexes

In [37]:
def create_text_search_index(collection, index_definition, index_name):
    """
    Create a search index for a MongoDB Atlas collection.

    Args:
    collection: MongoDB collection object
    index_definition: Dictionary defining the index mappings
    index_name: String name for the index

    Returns:
    str: Result of the index creation operation
    """

    try:
        search_index_model = SearchIndexModel(
            definition=index_definition, name=index_name
        )

        result = collection.create_search_index(model=search_index_model)
        print(f"Search index '{index_name}' created successfully")
        return result
    except Exception as e:
        print(f"Error creating search index: {e!s}")
        return None

In [38]:
employees_collection_search_index_defintion = {
    "mappings": {
        "dynamic": True,
        "fields": {
            "name": {"type": "string"},
            "role": {"type": "string"},
            "department": {"type": "string"},
            "bio": {"type": "string"},
        },
    }
}

projects_collection_search_index_defintion = {
    "mappings": {
        "dynamic": True,
        "fields": {
            "name": {"type": "string"},
            "description": {"type": "string"},
            "status": {"type": "string"},
        },
    }
}

faqs_collection_search_index_defintion = {
    "mappings": {
        "dynamic": True,
        "fields": {"question": {"type": "string"}, "answer": {"type": "string"}},
    }
}

knowledge_assets_collection_search_index_defintion = {
    "mappings": {
        "dynamic": True,
        "fields": {
            "title": {"type": "string"},
            "content": {"type": "string"},
            "type": {"type": "string"},
        },
    }
}

In [39]:
create_text_search_index(
    db[EMPLOYEES_COLLECTION],
    employees_collection_search_index_defintion,
    EMPLOYEES_SEARCH_INDEX_NAME,
)
create_text_search_index(
    db[FAQS_COLLECTION], faqs_collection_search_index_defintion, FAQS_SEARCH_INDEX_NAME
)
create_text_search_index(
    db[KNOWLEDGE_ASSETS_COLLECTION],
    knowledge_assets_collection_search_index_defintion,
    KNOWLEDGE_ASSETS_SEARCH_INDEX_NAME,
)
create_text_search_index(
    db[PROJECTS_COLLECTION],
    projects_collection_search_index_defintion,
    PROJECTS_SEARCH_INDEX_NAME,
)

Search index 'employees_text_search_index' created successfully
Search index 'faqs_text_search_index' created successfully
Search index 'knowledge_assets_text_search_index' created successfully
Search index 'projects_text_search_index' created successfully


'projects_text_search_index'

### Data Ingestion

In [40]:
employee_documents = employees_data_df.to_dict(orient="records")
faq_documents = faqs_data_df.to_dict(orient="records")
knowledge_asset_documents = knowledge_assets_data_df.to_dict(orient="records")
project_documents = projects_data_df.to_dict(orient="records")

db[EMPLOYEES_COLLECTION].insert_many(employee_documents)
db[FAQS_COLLECTION].insert_many(faq_documents)
db[KNOWLEDGE_ASSETS_COLLECTION].insert_many(knowledge_asset_documents)
db[PROJECTS_COLLECTION].insert_many(project_documents)

print(f"Inserted {len(employee_documents)} documents into {EMPLOYEES_COLLECTION}.")
print(f"Inserted {len(faq_documents)} documents into {FAQS_COLLECTION}.")
print(
    f"Inserted {len(knowledge_asset_documents)} documents into {KNOWLEDGE_ASSETS_COLLECTION}."
)
print(f"Inserted {len(project_documents)} documents into {PROJECTS_COLLECTION}.")

Inserted 10 documents into employees.
Inserted 5 documents into faqs.
Inserted 6 documents into knowledge_assets.
Inserted 8 documents into projects.


## Part 3: Creating and Testing Retrieval Methods With LangChain

### Text Search

In [41]:
# Test lexical search with MongoDB Atlas
from typing import Any, List, Tuple

from langchain.schema import Document
from langchain_mongodb.retrievers.full_text_search import (
    MongoDBAtlasFullTextSearchRetriever,
)


def full_text_search(
    collection, search_field, query: str, top_k: int = 10
) -> List[Document]:
    # Dynamically get the search index name from the collection name
    collection_name = collection.name
    search_index_name = f"{collection_name}_text_search_index"

    full_text_search = MongoDBAtlasFullTextSearchRetriever(
        collection=collection,
        search_index_name=search_index_name,
        search_field=search_field,
        k=top_k,
        include_scores=True,
    )
    result = full_text_search.get_relevant_documents(query)

    # Remove the emmbedding attribute from each document in the results
    for doc in result:
        del doc.metadata["embedding"]

    return result

In [42]:
full_text_search(
    collection=db[EMPLOYEES_COLLECTION], search_field="name", query="Jordan"
)

<ipython-input-41-952faf310953>:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = full_text_search.get_relevant_documents(query)


[Document(metadata={'_id': '68177f2b83be6a2352935af1', 'emp_id': 'employees-2', 'role': 'Network Engineer', 'department': 'Network Operations', 'skills': ['Network design', 'System troubleshooting', 'Routing and Switching', 'Telecommunications protocols', 'Fiber optic technology', 'Wireless communication'], 'bio': 'Jordan Lee is a skilled network engineer specialized in designing, maintaining, and optimizing telecommunications networks. With a strong background in both wired and wireless protocols, Jordan ensures high availability and performance for large-scale enterprise clients.', 'manager': 'employees-1', 'start_date': '2022-03-10', 'end_date': '', 'current_projects': [], 'past_projects': [], 'mentors': ['employees-0'], 'mentees': [], 'frequent_collaborators': ['employees-0', 'employees-1'], 'score': 0.8452645540237427}, page_content='Jordan Lee'),
 Document(metadata={'_id': '68177f2b83be6a2352935aef', 'emp_id': 'employees-0', 'role': 'Network Engineer', 'department': 'Telecommunic

### Vector Search

In [43]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_voyageai import VoyageAIEmbeddings

# Initialize embeddings model
embedding_model = VoyageAIEmbeddings(
    batch_size=1,
    model=VOYAGE_AI_EMBEDDING_MODEL,
    voyage_api_key=os.environ.get("VOYAGE_API_KEY"),
    output_dimension=VOYAGE_AI_EMBEDDING_MODEL_DIMENSION,
    show_progress_bar=True,
)


def semantic_search(
    collection, text_key: str, query: str, top_k: int = 10
) -> List[Tuple[Any, float]]:
    # Dynamically get the vector search index name from the collection name
    collection_name = collection.name
    vector_search_index_name = f"{collection_name}_vector_search_index"

    vector_store = MongoDBAtlasVectorSearch.from_connection_string(
        connection_string=os.environ.get("MONGODB_URI"),
        namespace=f"{DB_NAME}.{collection_name}",
        embedding=embedding_model,
        index_name=vector_search_index_name,
        text_key=text_key,
    )

    return vector_store.similarity_search_with_score(query=query, k=top_k)

In [44]:
semantic_search(
    collection=db[EMPLOYEES_COLLECTION],
    text_key="bio",
    query="Get me someone that is good at speaking to clients",
    top_k=3,
)

[(Document(id='6818f00a6c0be1dbff0f6d88', metadata={'_id': '6818f00a6c0be1dbff0f6d88', 'emp_id': 'employees-4', 'name': 'Samantha Lee', 'role': 'Project Manager', 'department': 'Network Operations', 'skills': ['project management', 'telecommunications', 'fiber optic networks', 'agile methodologies', 'stakeholder communication', 'budget management'], 'manager': 'employees-1', 'start_date': '2022-02-15', 'end_date': '', 'current_projects': [], 'past_projects': [], 'mentors': ['employees-1'], 'mentees': ['employees-2'], 'frequent_collaborators': ['employees-0', 'employees-3']}, page_content='Samantha Lee is an experienced Project Manager in the telecommunications field, specializing in leading cross-functional teams for the deployment and maintenance of large-scale network solutions. With strong skills in both agile and traditional project management, Samantha ensures timely delivery of projects while exceeding quality standards.'),
  0.7237071990966797),
 (Document(id='6818f00a6c0be1dbff

### Hybrid Search

In [45]:
from langchain_mongodb.retrievers import MongoDBAtlasHybridSearchRetriever


def hybrid_search(
    collection, text_key: str, query: str, top_k: int = 10
) -> List[Document]:
    # Dynamically get the vector search index name from the collection name
    collection_name = collection.name
    vector_search_index_name = f"{collection_name}_vector_search_index"
    search_index_name = f"{collection_name}_text_search_index"

    # intilaize the vector store first
    vector_store = MongoDBAtlasVectorSearch.from_connection_string(
        connection_string=os.environ.get("MONGODB_URI"),
        namespace=f"{DB_NAME}.{collection_name}",
        embedding=embedding_model,
        index_name=vector_search_index_name,
        text_key="bio",
    )

    hybrid_search = MongoDBAtlasHybridSearchRetriever(
        vectorstore=vector_store, search_index_name=search_index_name, top_k=top_k
    )

    return hybrid_search.get_relevant_documents(query)

In [46]:
hybrid_search(
    collection=db[EMPLOYEES_COLLECTION],
    text_key="bio",
    query="Get me someone that know android development",
    top_k=3,
)

[Document(metadata={'_id': '6818c9e80373b444e9b5d6d3', 'emp_id': 'employees-4', 'name': 'Anjali Kapoor', 'role': 'Network Engineer', 'department': 'Network Operations', 'skills': ['Fiber Optic Networks', 'Network Troubleshooting', '4G/5G Technologies', 'IP Routing', 'Cisco Devices', 'Wireless Communications'], 'manager': 'employees-1', 'start_date': '2019-08-12', 'end_date': '', 'current_projects': [], 'past_projects': [], 'mentors': ['employees-0'], 'mentees': ['employees-2'], 'frequent_collaborators': ['employees-3'], 'vector_score': 0.01639344262295082, 'rank': 0, 'fulltext_score': 0, 'score': 0.01639344262295082}, page_content='Anjali is a skilled Network Engineer with 6 years of experience in telecommunication infrastructure. She specializes in designing and maintaining high-availability networks for mobile operators, ensuring seamless customer connectivity.'),
 Document(metadata={'_id': '6818cf240373b444e9b5d6f7', 'emp_id': 'employees-4', 'name': 'Anjali Kapoor', 'role': 'Network

### Graph Search

In [47]:
from langchain.chat_models import init_chat_model
from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from langchain_openai import OpenAI

# For best results, use latest models such as gpt-4o and Claude Sonnet 3.5+, etc.
chat_model = init_chat_model("gpt-4o", model_provider="openai", temperature=0)


def graph_traversal(collection, query):
    """
    Execute a Graph RAG query against a MongoDB collection.

    Args:
    collection: MongoDB collection object
    query: String query to execute

    Returns:
    str: Result of the query execution

    """

    collection_name = collection.name

    graph_store = MongoDBGraphStore(
        connection_string=os.environ.get("MONGODB_URI"),
        database_name=DB_NAME,
        collection_name=collection_name,
        entity_extraction_model=chat_model,
    )

    results = graph_store.chat_response(query)

    return results

In [48]:
graph_traversal(
    collection=db[PROJECTS_COLLECTION],
    query="Find all projects that share team members with good communication",
)

AIMessage(content='There are no entities related to projects or team members with good communication in the retrieved information.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 472, 'total_tokens': 491, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BU9w93V2fXz2ZZf5QHazWEnamV27b', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c949b608-3ad4-4656-8c4f-e6bc3f670600-0', usage_metadata={'input_tokens': 472, 'output_tokens': 19, 'total_tokens': 491, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Cross-Team Project Knowledge Discovery

- Finding how different projects interconnect through shared team members
- Identifying knowledge transfer paths when employees move between projects
- Discovering dependencies between projects that aren't documented but exist through shared personnel


Expert Network Mapping

- Tracing expertise flows when experts collaborate on projects
- Finding indirect expertise paths (e.g., "Who can John reach out to for
Android development help through his network?")
- Discovering emerging expertise clusters around specific technologies

## Part 4: Automated Workflow and Agentic AI Implementation

### Automated Workflow Scenarios for Enterprise Knowledge Discovery

TODO: Define workflow here




#### **SCENARIO 1: Critical 5G Network Issue Response (Workflow)**
    
- Context: A major 5G network outage affects multiple regions. The system needs to
quickly assemble an emergency response team with specific expertise.

- Workflow Steps:
  - Step 1: Crisis Detection and Skill Requirements
  - Step 2: Expert Identification
  - Step 3: Team Composition Analysis
  - Step 4: Knowledge Asset Preparation
  - Step 5: Team Activation and Brief


Creating an crisis event collection with search and vector indexes

In [49]:
CRISIS_EVENT_COLLECTION = "crisis_events"

existing_collections = db.list_collection_names()

# Create collection
if CRISIS_EVENT_COLLECTION not in existing_collections:
    db.create_collection(CRISIS_EVENT_COLLECTION)
    print(f"Created {CRISIS_EVENT_COLLECTION} collection")
else:
    print(f"{CRISIS_EVENT_COLLECTION} collection already exists")

crisis_events collection already exists


In [50]:
# Create search and vector indexes
create_vector_search_index(
    db[CRISIS_EVENT_COLLECTION], f"{CRISIS_EVENT_COLLECTION}_vector_search_index"
)
create_text_search_index(
    db[CRISIS_EVENT_COLLECTION],
    {"mappings": {"dynamic": True}},
    f"{CRISIS_EVENT_COLLECTION}_text_search_index",
)

Vector search index 'crisis_events_vector_search_index' already exists.
Search index 'crisis_events_text_search_index' created successfully


'crisis_events_text_search_index'

Creating data models for crisis events

In [51]:
from datetime import datetime
from enum import Enum


# Minimal enums
class CrisisType(str, Enum):
    NETWORK_OUTAGE = "Network Outage"
    SECURITY_BREACH = "Security Breach"
    SYSTEM_FAILURE = "System Failure"
    INFRASTRUCTURE_FAILURE = "Infrastructure Failure"


class SeverityLevel(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"


# Minimal CrisisEvent Model
class CrisisEvent(BaseModel):
    event_id: str = Field(..., description="Unique crisis identifier")
    event_type: CrisisType = Field(..., description="Type of crisis")
    severity: SeverityLevel = Field(..., description="Crisis severity level")
    title: str = Field(..., description="Brief crisis description")
    description: str = Field(..., description="Detailed crisis description")
    affected_systems: List[str] = Field(
        default_factory=list, description="Affected systems/services"
    )
    affected_regions: List[str] = Field(
        default_factory=list, description="Affected geographical regions"
    )
    customer_impact: str = Field(
        ..., description="Estimated customer impact description"
    )
    required_skills: List[str] = Field(
        default_factory=list, description="Skills needed for response"
    )

Creating an incident report parser

In [63]:
class IncidentReportParser:
    def parse_incident_report(self, incident_report: str) -> CrisisEvent:
        """Parse incident report text and create CrisisEvent object"""

        # Create a prompt for GPT-4.1 to parse the incident report
        prompt = f"""
    Parse the following incident report and extract information to create a CrisisEvent.

    Incident Report:
    {incident_report}

    Extract the following information:
    1. event_type: Determine the type (Network Outage, Security Breach, System Failure, Infrastructure Failure)
    2. severity: Determine severity level (low, medium, high, critical)
    3. title: Create a brief title (max 100 characters)
    4. description: Extract or create a detailed description (max 500 characters)
    5. affected_systems: List of affected systems/services
    6. affected_regions: List of affected geographical regions
    7. customer_impact: Estimated impact on customers
    8. required_skills: Skills needed to respond to this crisis

    Generate a unique event_id in the format: CRISIS-YYYYMMDD-XXX

    Return ONLY a JSON object that matches the CrisisEvent schema exactly.
    """

        try:
            # Parse with GPT-4.1
            response = openai_client.responses.parse(
                model="gpt-4.1", input=prompt, text_format=CrisisEvent
            )

            crisis_event = response.output_parsed

            return crisis_event

        except Exception as e:
            print(f"Error parsing incident report: {e}")
            # Fallback to basic crisis event
            return self._create_fallback_crisis(incident_report)

    def _create_fallback_crisis(self, incident_report: str) -> CrisisEvent:
        """Create a basic crisis event if parsing fails"""
        today = datetime.now()
        event_id = f"CRISIS-{today.strftime('%Y%m%d')}-001"

        return CrisisEvent(
            event_id=event_id,
            event_type=CrisisType.SYSTEM_FAILURE,
            severity=SeverityLevel.MEDIUM,
            title="Unknown Crisis Event",
            description=incident_report[:500],  # Truncate to 500 chars
            affected_systems=["Unknown"],
            affected_regions=["Unknown"],
            customer_impact="Assessment required",
            required_skills=["General IT Support"],
        )

In [64]:
# Example incident report (single string)
incident_report = """
  NETWORK CRISIS REPORT - PRIORITY CRITICAL

  Incident #: INC-20250505-3547
  Service: 5G Network Service
  Status: ACTIVE OUTAGE

  SUMMARY:
  Complete 5G network failure reported across North America region

  AFFECTED AREAS:
  - New York City metro area
  - Boston metropolitan region
  - Philadelphia and surrounding counties

  IMPACT ASSESSMENT:
  - Estimated 2 million customers unable to access 5G services
  - Enterprise customers reporting business-critical service disruptions
  - Mobile data speeds degraded to 4G in surrounding areas

  TECHNICAL DETAILS:
  - Core Network Status: DOWN
  - gNodeB Stations: 3/5 nodes failed
  - Data Center: Primary facility shows hardware failures
  - Root Cause: Equipment overheating during maintenance window

  TIMELINE:
  15:00 EST - Maintenance window begins
  15:25 EST - First customer complaints received
  15:30 EST - Network monitoring alerts triggered
  15:45 EST - Service outage confirmed

  REQUIRED RESPONSE:
  - Network engineers with 5G expertise
  - Hardware repair technicians
  - Crisis management team
  - Customer communications team

  BUSINESS IMPACT:
  - Revenue impact: $5,000/minute
  - SLA breach: Yes (2-hour response requirement)
  - Media attention: High (local news coverage)

  NEXT STEPS:
  1. Activate emergency response protocol
  2. Dispatch on-site technicians
  3. Prepare customer communications
  4. Assess backup systems deployment
"""

In [65]:
parser = IncidentReportParser()

# Parse incident report
print("=== Processing Incident Report ===")

# Parse incident report into CrisisEvent
crisis_event = parser.parse_incident_report(incident_report)

# Display results
print(f"Event ID: {crisis_event.event_id}")
print(f"Type: {crisis_event.event_type}")
print(f"Severity: {crisis_event.severity}")
print(f"Title: {crisis_event.title}")
print(f"Description: {crisis_event.description}")
print(f"Affected Systems: {', '.join(crisis_event.affected_systems)}")
print(f"Affected Regions: {', '.join(crisis_event.affected_regions)}")
print(f"Customer Impact: {crisis_event.customer_impact}")
print(f"Required Skills: {', '.join(crisis_event.required_skills)}")
print()

=== Processing Incident Report ===
Event ID: CRISIS-20250505-001
Type: CrisisType.NETWORK_OUTAGE
Severity: SeverityLevel.CRITICAL
Title: Critical 5G Network Failure Impacting North America Metro Regions
Description: A complete failure of the 5G network service has rendered millions of customers in major North American metro areas unable to access 5G. Enterprise clients report major disruptions, with core network and primary data center hardware failures traced to overheating during scheduled maintenance.
Affected Systems: 5G Network Service, Core Network, gNodeB Stations, Primary Data Center
Affected Regions: New York City metro area, Boston metropolitan region, Philadelphia and surrounding counties
Customer Impact: About 2 million customers offline; enterprise clients facing business-critical disruptions; slowed mobile data for surrounding areas.
Required Skills: 5G network engineering, Hardware repair, Crisis management, Customer communications



In [159]:
class IssueResponseEngine:
    def __init__(self, db):
        self.db = db

    def crisis_detection_and_parsing(self, incident_report):
        """Integrates incident report parsing with emergency response"""

        # Initialize a parser
        parser = IncidentReportParser()

        # Parse incident into CrisisEvent
        crisis_event = parser.parse_incident_report(incident_report)

        # Save to MongoDB
        db.crisis_events.insert_one(crisis_event.model_dump())
        print("Crisis event saved into records")

        # Initialize emergency response
        crisis_data = crisis_event.model_dump()

        print("Crisis Event Generated:")
        print(json.dumps(crisis_event.model_dump(), indent=2))

        return crisis_data

    def experts_identification(self, crisis_data, limit=5):
        """Identifies experts with required skills"""
        # Use the skills data in the crisis_data to search the employees for the right skills
        skills_to_search_against = crisis_data["required_skills"]

        # Create search query focusing on the skills array and bio fields
        search_query = f"Find experts with {', '.join(skills_to_search_against)} in their skills and experience"

        print(f"Search Query: {search_query}")

        # Use hybrid search to retrieve employees with the right skills
        results = hybrid_search(
            collection=db[EMPLOYEES_COLLECTION],
            text_key="bio",
            query=search_query,
            top_k=limit,
        )

        return results

    def knowledge_asset_gathering(self, crisis_data, limit=5):
        """Gather relevant knowledge assets for the team"""

        # Look for knowledge assets that are semantically similar to the crisis event description
        search_query = crisis_data["description"]

        # Use semantic search to retrieve knowledge assets
        results = semantic_search(
            collection=self.db[KNOWLEDGE_ASSETS_COLLECTION],
            text_key="content",
            query=search_query,
            top_k=limit,
        )

        return results

    def team_activation_and_brief(
        self, crisis_data, experts_identified, knowledge_assets
    ):
        """Create response plan and activate team"""

        try:
            # Prepare the prompt with crisis and team information
            prompt = f"""
      CRISIS EVENT BRIEFING

      Crisis Details:
      - Event ID: {crisis_data.get('event_id')}
      - Type: {crisis_data.get('event_type')}
      - Severity: {crisis_data.get('severity')}
      - Title: {crisis_data.get('title')}
      - Description: {crisis_data.get('description')}
      - Affected Systems: {', '.join(crisis_data.get('affected_systems', []))}
      - Affected Regions: {', '.join(crisis_data.get('affected_regions', []))}
      - Customer Impact: {crisis_data.get('customer_impact')}

      Response Team:
      {self._format_team_members(experts_identified)}

      Relevant Knowledge Assets:
      {self._format_knowledge_assets(knowledge_assets)}

      INSTRUCTIONS:
      Create a detailed briefing for the emergency response team that includes:
      1. Executive summary of the crisis
      2. Team assignments with specific roles
      3. Priority action items
      4. Available resources and documentation
      5. Expected timeline and milestones
      6. Communication protocols
      7. Success criteria for resolution

      Keep the briefing concise but comprehensive.
      """

            # Call GPT-4.1 to generate briefing
            response = openai_client.responses.create(
                model="gpt-4.1",
                input=prompt,
            )

            briefing_text = response.output_text

            print("Briefing Generated:")
            print(briefing_text)

            return briefing_text

        except Exception as e:
            print(f"Error generating briefing: {e}")

    def _format_team_members(self, experts):
        """Format expert list for briefing"""
        formatted = []
        for i, expert in enumerate(experts):
            role = self._assign_crisis_role(expert)
            formatted.append(
                f"- {expert.get('name')} ({expert.get('role')}) - Crisis Role: {role}"
            )
        return "\n".join(formatted)

    def _format_knowledge_assets(self, assets):
        """Format knowledge assets for briefing"""
        if not assets:
            return "No specific knowledge assets found."
        formatted = []
        for asset in assets[:5]:  # Limit to top 5
            formatted.append(f"- {asset.get('title')} (Type: {asset.get('type')})")
        return "\n".join(formatted)

    def _assign_crisis_role(self, expert):
        """Assign crisis-specific role based on expert skills"""
        skills = expert.get("matching_skills", [])
        if any("5G" in skill for skill in skills):
            return "Technical Lead"
        elif any("Security" in skill for skill in skills):
            return "Security Specialist"
        elif any("Network" in skill for skill in skills):
            return "Network Specialist"
        else:
            return "Technical Support"

In [160]:
from datetime import datetime
from typing import Any, Dict, List, TypedDict, Union

import langgraph.graph as lg
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.mongodb import MongoDBSaver


# Define state for the crisis response graph
class EmergencyResponseState(TypedDict):
    """State for the emergency response workflow"""

    incident_report: str  # Incident report text
    crisis_event: Dict[str, Any]  # Crisis details
    skill_requirements: Optional[List[str]]  # Required skills
    available_experts: Optional[List[Dict[str, Any]]]  # Identified experts
    selected_team: Optional[List[Dict[str, Any]]]  # Final team composition
    relevant_knowledge: Optional[List[Dict[str, Any]]]  # Knowledge assets
    response_plan: Optional[Dict[str, Any]]  # Final response plan
    messages: List[Union[HumanMessage, AIMessage]]  # Conversation history
    errors: Optional[List[str]]  # Any errors encountered

In [161]:
class EmergencyResponseWorkflow:
    """Orchestrates the emergency response workflow using LangGraph"""

    def __init__(self, client, db, collection_names):
        self.client = client
        self.db = db
        self.employees_collection = db[collection_names["employees"]]
        self.projects_collection = db[collection_names["projects"]]
        self.knowledge_collection = db[collection_names["knowledge_assets"]]

        # Initialize IssueResponseEngine
        self.issue_engine = IssueResponseEngine(db)

        # Create MongoDB checkpointer
        self.checkpoint_store = MongoDBSaver(client, DB_NAME, "checkpoints")

        # Build the graph
        self.workflow = self._build_graph()

    def _detect_crisis_and_requirements(
        self, state: EmergencyResponseState
    ) -> EmergencyResponseState:
        """Detect crisis parameters and determine skill requirements"""
        print("\n")
        print("1. Beginning crisis detecting and parsing provided information...")
        try:
            incident_report = state["incident_report"]

            # Use IssueResponseEngine to parse incident and create crisis data
            crisis_data = self.issue_engine.crisis_detection_and_parsing(
                incident_report
            )

            state["crisis_event"] = crisis_data

            # Update skill requirements from crisis data
            state["skill_requirements"] = crisis_data.get("required_skills", [])

            state["messages"].append(
                AIMessage(
                    content=f"Crisis detected: {crisis_data.get('event_type')}. Severity: {crisis_data.get('severity')}. "
                    f"Required skills identified: {', '.join(state['skill_requirements'])}"
                )
            )

        except Exception as e:
            error_msg = f"Error detecting crisis requirements: {e!s}"
            state["errors"] = state.get("errors", []) + [error_msg]
            state["messages"].append(AIMessage(content=error_msg))

        return state

    def _identify_experts(
        self, state: EmergencyResponseState
    ) -> EmergencyResponseState:
        """Identify available experts with required skills"""
        print("\n")
        print(
            "2. Identifying experts within records suitable to handle crisis event..."
        )
        try:
            # Use IssueResponseEngine to identify experts
            crisis_data = state["crisis_event"]
            available_experts = self.issue_engine.experts_identification(crisis_data, 5)

            # Format experts for consistency with existing state structure
            formatted_experts = []
            for expert in available_experts:
                expert = expert.metadata
                formatted_experts.append(
                    {
                        "emp_id": expert.get("emp_id"),
                        "name": expert.get("name"),
                        "role": expert.get("role"),
                        "department": expert.get("department"),
                        "bio": expert.get("bio"),
                        "skills": expert.get("skills", []),
                        "current_projects": expert.get("current_projects", []),
                    }
                )

            print("Below are the experts identified ⬇️")
            print(formatted_experts)

            # The available experts are the selected team
            # TODO: Create a process that organizes the team based on the available experts
            state["selected_team"] = formatted_experts
            state["messages"].append(
                AIMessage(
                    content=f"Identified {len(formatted_experts)} available experts with required skills."
                )
            )

        except Exception as e:
            print(e)
            error_msg = f"Error identifying experts: {e!s}"
            # Fix for the second error
            if state.get("errors") is None:
                state["errors"] = [error_msg]
            else:
                state["errors"] = state.get("errors", []) + [error_msg]
            state["messages"].append(AIMessage(content=error_msg))

        return state

    def _gather_knowledge_assets(
        self, state: EmergencyResponseState
    ) -> EmergencyResponseState:
        """Gather relevant knowledge assets for the team"""
        print("\n")
        print("3. Gathering knowledge assets to prep team on...")

        try:
            # Use IssueResponseEngine to gather knowledge assets
            crisis_data = state["crisis_event"]
            knowledge_assets = self.issue_engine.knowledge_asset_gathering(crisis_data)
            print("Below are the knowledge assets gathered ⬇️")

            # Format knowledge assets for consistency
            formatted_assets = []
            for asset in knowledge_assets:
                # Handle tuple format (Document, score)
                if isinstance(asset, tuple):
                    doc = asset[0]
                    if hasattr(doc, "metadata"):
                        asset_data = doc.metadata
                    else:
                        asset_data = doc
                elif hasattr(asset, "metadata"):
                    asset_data = asset.metadata
                else:
                    asset_data = asset

                formatted_assets.append(
                    {
                        "asset_id": asset_data.get("asset_id", "unknown"),
                        "title": asset_data.get("title", "Untitled Asset"),
                        "type": asset_data.get("type", "documentation"),
                        "author": asset_data.get("author", "Unknown"),
                        "content": asset_data.get("content", ""),
                        "creation_date": asset_data.get("creation_date", ""),
                    }
                )

            print(formatted_assets)

            state["relevant_knowledge"] = formatted_assets
            state["messages"].append(
                AIMessage(
                    content=f"Gathered {len(formatted_assets)} relevant knowledge assets for the team."
                )
            )

        except Exception as e:
            print("Error gethering knowledge assets ❌")
            print(e)
            error_msg = f"Error gathering knowledge assets: {e!s}"
            state["errors"] = state.get("errors", []) + [error_msg]
            state["messages"].append(AIMessage(content=error_msg))

        return state

    def _activate_team_and_create_plan(
        self, state: EmergencyResponseState
    ) -> EmergencyResponseState:
        """Create response plan and activate team"""
        print("4. Activating team and creating a response plan...")

        try:
            # TODO: Send an email to all employees selected and include response plan
            selected_team = state["selected_team"]
            crisis_event = state["crisis_event"]
            relevant_knowledge = state["relevant_knowledge"]

            # Use IssueResponseEngine to create team briefing
            briefing_text = self.issue_engine.team_activation_and_brief(
                crisis_event, selected_team, relevant_knowledge
            )

            # Create response plan
            response_plan = {
                "crisis_id": f"CRISIS-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
                "team_lead": selected_team[0] if selected_team else None,
                "team_members": selected_team,
                "crisis_details": crisis_event,
                "briefing": briefing_text,
                "action_items": self._generate_action_items(
                    crisis_event, selected_team
                ),
                "knowledge_resources": relevant_knowledge,
                "status": "active",
                "created_at": datetime.now().isoformat(),
                "expected_resolution_time": self._estimate_resolution_time(
                    crisis_event
                ),
            }

            # Create summary for team activation
            activation_summary = self._create_activation_summary(response_plan)

            state["response_plan"] = response_plan
            state["messages"].append(
                AIMessage(
                    content=f"Emergency response team activated. Plan created: {activation_summary}"
                )
            )

        except Exception as e:
            error_msg = f"Error activating team: {e!s}"
            state["errors"] = state.get("errors", []) + [error_msg]
            state["messages"].append(AIMessage(content=error_msg))

        return state

    def _estimate_resolution_time(self, crisis_event: Dict[str, Any]) -> str:
        """Estimate resolution time based on crisis severity"""
        severity = crisis_event.get("severity", "low")
        if severity == "critical":
            return "1-2 hours"
        elif severity == "high":
            return "4-8 hours"
        else:
            return "24-48 hours"

    def _create_activation_summary(self, response_plan: Dict[str, Any]) -> str:
        """Create summary for team activation"""
        team_size = len(response_plan["team_members"])
        team_lead = (
            response_plan["team_lead"]["name"] if response_plan["team_lead"] else "None"
        )
        crisis_type = response_plan["crisis_details"].get("event_type", "Unknown")
        resolution_time = response_plan["expected_resolution_time"]

        return (
            f"Crisis {response_plan['crisis_id']}: {crisis_type} response team activated. "
            f"Team of {team_size} led by {team_lead}. Expected resolution: {resolution_time}"
        )

    def _should_continue(self, state: EmergencyResponseState) -> str:
        """Determine if workflow should continue or end"""
        if state.get("errors") and len(state["errors"]) < 3:
            # Retry if we have minor errors
            print("There were minor errors so retrying...")
            print(state["errors"])
            return "retry"
        elif not state.get("selected_team"):
            # End if no team could be formed
            return "end_failure"
        else:
            # Continue to completion
            return "end_success"

    def _build_graph(self):
        """Build the LangGraph workflow"""
        # Define the graph
        builder = lg.StateGraph(EmergencyResponseState)

        # Add nodes
        builder.add_node("detect_crisis", self._detect_crisis_and_requirements)
        builder.add_node("identify_experts", self._identify_experts)
        builder.add_node("gather_knowledge", self._gather_knowledge_assets)
        builder.add_node("activate_team", self._activate_team_and_create_plan)

        # Define edges
        builder.add_edge("detect_crisis", "identify_experts")
        builder.add_edge("identify_experts", "gather_knowledge")
        builder.add_edge("gather_knowledge", "activate_team")

        # Add conditional edge for completion
        builder.add_conditional_edges(
            "activate_team",
            self._should_continue,
            {"retry": "detect_crisis", "end_success": lg.END, "end_failure": lg.END},
        )

        # Set entry point
        builder.set_entry_point("detect_crisis")

        # Compile the graph with MongoDB checkpointing
        return builder.compile(checkpointer=self.checkpoint_store)

    def respond_to_crisis(self, incident_report) -> Dict[str, Any]:
        """Respond to an emergency crisis event"""
        # Initialize state
        initial_state = EmergencyResponseState(
            incident_report=incident_report,
            crisis_event={},
            skill_requirements=None,
            available_experts=None,
            selected_team=None,
            relevant_knowledge=None,
            response_plan=None,
            messages=[HumanMessage(content=incident_report)],
            errors=[],
        )

        # Run the workflow
        config = {"configurable": {"thread_id": 1}}
        final_state = self.workflow.invoke(initial_state, config)

        return final_state

In [162]:
# Initialize the workflow
collection_names = {
    "employees": EMPLOYEES_COLLECTION,
    "projects": PROJECTS_COLLECTION,
    "knowledge_assets": KNOWLEDGE_ASSETS_COLLECTION,
}

incident_report = """
  NETWORK CRISIS REPORT - PRIORITY CRITICAL

  Incident #: INC-20250505-3547
  Service: 5G Network Service
  Status: ACTIVE OUTAGE

  SUMMARY:
  Complete 5G network failure reported across North America region

  AFFECTED AREAS:
  - New York City metro area
  - Boston metropolitan region
  - Philadelphia and surrounding counties

  IMPACT ASSESSMENT:
  - Estimated 2 million customers unable to access 5G services
  - Enterprise customers reporting business-critical service disruptions
  - Mobile data speeds degraded to 4G in surrounding areas

  TECHNICAL DETAILS:
  - Core Network Status: DOWN
  - gNodeB Stations: 3/5 nodes failed
  - Data Center: Primary facility shows hardware failures
  - Root Cause: Equipment overheating during maintenance window

  TIMELINE:
  15:00 EST - Maintenance window begins
  15:25 EST - First customer complaints received
  15:30 EST - Network monitoring alerts triggered
  15:45 EST - Service outage confirmed

  REQUIRED RESPONSE:
  - Network engineers with 5G expertise
  - Hardware repair technicians
  - Crisis management team
  - Customer communications team

  BUSINESS IMPACT:
  - Revenue impact: $5,000/minute
  - SLA breach: Yes (2-hour response requirement)
  - Media attention: High (local news coverage)

  NEXT STEPS:
  1. Activate emergency response protocol
  2. Dispatch on-site technicians
  3. Prepare customer communications
  4. Assess backup systems deployment
"""

emergency_workflow = EmergencyResponseWorkflow(db_client, db, collection_names)

# Execute the complete workflow starting from incident report
print("=== Emergency Response System Activated ===")
result = emergency_workflow.respond_to_crisis(
    incident_report
)  # Empty dict to trigger detection & parsing

# Print results
if result.get("response_plan"):
    print(f"\nCrisis ID: {result['response_plan']['crisis_id']}")
    print(f"Team Lead: {result['response_plan']['team_lead']['name']}")
    print(f"Team Size: {len(result['response_plan']['team_members'])}")
    print(f"Expected Resolution: {result['response_plan']['expected_resolution_time']}")

    print("\nTeam Composition:")
    for member in result["response_plan"]["team_members"]:
        print(f"- {member['name']} ({member['assigned_role']})")

    print("\nAction Items:")
    for action in result["response_plan"]["action_items"]:
        print(f"- {action['title']} (Priority: {action['priority']})")

    print("\nBriefing:")
    print(result["response_plan"]["briefing"])
else:
    print("Failed to create emergency response plan")

=== Emergency Response System Activated ===


1. Beginning crisis detecting and parsing provided information...
Crisis event saved into records
Crisis Event Generated:
{
  "event_id": "CRISIS-20250505-001",
  "event_type": "Network Outage",
  "severity": "critical",
  "title": "Critical 5G Network Outage Across Major North American Cities",
  "description": "A complete 5G network failure has occurred across New York City, Boston, and Philadelphia, impacting core network systems and major data center hardware. 2 million customers lose 5G access; enterprise clients face business-critical disruption. Outage caused by equipment overheating during maintenance.",
  "affected_systems": [
    "5G Network Service",
    "Core Network",
    "gNodeB Stations",
    "Primary Data Center"
  ],
  "affected_regions": [
    "New York City metro area",
    "Boston metropolitan region",
    "Philadelphia and surrounding counties"
  ],
  "customer_impact": "2 million customers unable to access 5G; enterpri

#### **SCENARIO 2: Complete Employee Departure Process (Agentic AI)**
    
- Context: A senior network engineer has announced their departure.The system orchestrates the entire offboarding process, including knowledge transfer, project handover, and official record updates.

- Workflow Steps:
  - Step 1: Departure Announcement and Initial Setup
  - Step 2: Critical Knowledge and Project Assessment
  - Step 3: Project Transition Planning
  - Step 4: Knowledge Transfer Process
  - Step 5: Final Employee Record Update
